# Catch Basin Classifier

## Imports

In [54]:
import requests
import cv2
import tensorflow as tf
from pathlib import Path
import pandas as pd
from object_detection.utils import dataset_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

## Load the Model

In [41]:
MODEL_NAME = "ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
DOWNLOAD_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz"

In [6]:
print("Downloading model...")
r = requests.get(DOWNLOAD_URL)
with open(MODEL_NAME + ".tar.gz", "wb") as f:
    f.write(r.content)
print("Model downloaded.")

Model downloaded.


In [7]:
!tar -xf {MODEL_NAME + ".tar.gz"}

In [42]:
model = tf.saved_model.load(str(Path(MODEL_NAME, "saved_model").absolute()))

In [44]:
model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f9d8b920a60>

## Label Map
`label_map.pbtxt` maps the class (name of number) to a number. 

In [22]:
!cat label_map.pbtxt

item {
    id: 1
    name: 'blocked'
}
item {
    id: 2
    name: 'partial'
}
item {
    id: 3
    name: 'clear'
}

Define some utility functions to convert from class to `int` and `int` to class.

In [29]:
def class_to_int(class_name):
    if class_name == "blocked":
        return 1
    elif class_name == "partial":
        return 2
    elif class_name == "clear":
        return 3
    else:
        raise Exception("Invalid input")


def int_to_class(integer):
    if integer == 1:
        return "blocked"
    elif integer == 2:
        return "partial"
    elif integer == 3:
        return "clear"
    else:
        raise Exception("Invalid input")

## Prepare data

Convert all PASCAL VOC (XML) files in `data/` to a CSV file, `labels.csv`

In [8]:
!python xml_labels_to_csv.py

Converting to CSV...
Done.


Load the CSV file with Pandas

In [4]:
df = pd.read_csv("labels.csv")
df.head()

,filename,class,width,height,xmin,ymin,xmax,ymax
0,B1.JPG,blocked,659,800,316,550,458,609
1,B2.JPG,blocked,574,716,351,591,436,647
2,B3.JPG,blocked,576,792,220,466,365,517
3,B4.JPG,blocked,505,695,159,398,343,427
4,C1.JPG,clear,585,765,293,442,396,473


Split data into test data and train data

In [9]:
train_df = df.sample(frac=0.8, random_state=100)
test_df = df.drop(train_df.index).sample(frac=1.0)

Create TF Records within `annotations/` directory.

In [55]:
if not Path("annotations").exists():
    !mkdir annotations

def create_tf_record(dataframe, record_filename):
    with tf.io.TFRecordWriter(str(Path("annotations", record_filename))) as writer:
        for index, row in dataframe.iterrows():
            filename_encoded = row["filename"].encode("utf-8")
            width = int(row["width"])
            height = int(row["height"])
            encoded_jpg = None
            with tf.io.gfile.GFile(str(Path("data") / row["filename"]), "rb") as f:
                encoded_jpg = f.read()
            if encoded_jpg is None:
                raise Exception("Unable to read image: " + row["filename"])

            example = tf.train.Example(features=tf.train.Features(feature={
                "image/height": dataset_util.int64_feature(height),
                "image/width": dataset_util.int64_feature(width),
                "image/filename": dataset_util.bytes_feature(filename_encoded),
                "image/source_id": dataset_util.bytes_feature(filename_encoded),
                "image/encoded": dataset_util.bytes_feature(encoded_jpg),
                "image/format": dataset_util.bytes_feature(b"jpg"),
                "image/object/bbox/xmin": dataset_util.float_list_feature([int(row["xmin"]) / width]),
                "image/object/bbox/xmax": dataset_util.float_list_feature([int(row["xmax"]) / width]),
                "image/object/bbox/ymin": dataset_util.float_list_feature([int(row["ymin"]) / height]),
                "image/object/bbox/ymax": dataset_util.float_list_feature([int(row["ymax"]) / height]),
                "image/object/class/text": dataset_util.bytes_list_feature([row["class"].encode("utf-8")]),
                "image/object/class/label": dataset_util.int64_list_feature([class_to_int(row["class"])]),
            }))
            writer.write(example.SerializeToString())

# Train Data
create_tf_record(train_df, "train.record")
# Test Data
create_tf_record(test_df, "test.record")

## Edit Model Configuration
Edit `pipeline.config` to configure the model to be better for the data.

Load `pipeline.config` into a python object.

In [53]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(str(Path(MODEL_NAME, "pipeline.config")), "r") as f:
    text_format.Merge(f.read(), pipeline_config)

Edit attributes of `pipeline_config`.

In [61]:
# 3 Classes: blocked, partial, and clear.
pipeline_config.model.ssd.num_classes = 3
# Set batch_size based on memory available.
pipeline_config.train_config.batch_size = 4
# Path to checkpoint of model
pipeline_config.train_config.fine_tune_checkpoint = str(Path(MODEL_NAME, "checkpoint", "ckpt-0"))
pipeline_config.train_input_reader.label_map_path = "label_map.pbtxt"
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [str(Path("annotations", "train.record"))]
pipeline_config.eval_input_reader[0].label_map_path = "label_map.pbtxt"
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [str(Path("annotations", "test.record"))]

Save `pipeline_config` to `pipeline.config`.

In [62]:
with tf.io.gfile.GFile(str(Path(MODEL_NAME, "pipeline.config")), "w") as f:
    config_text = text_format.MessageToString(pipeline_config)
    f.write(config_text)

## Train the Model

In [66]:
!python tensorflow/models/research/object_detection/model_main_tf2.py --model_dir={MODEL_NAME} --pipeline_config_path={MODEL_NAME}/pipeline.config

2021-11-08 21:39:57.299899: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/snksynthesis/.local/lib/python3.8/site-packages/cv2/../../lib64:
2021-11-08 21:39:57.299941: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-11-08 21:40:01.859118: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/snksynthesis/.local/lib/python3.8/site-packages/cv2/../../lib64:/home/snksynthesis/.local/lib/python3.8/site-packages/cv2/../../lib64:
2021-11-08 21:40:01.859163: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-08 21:40:01

## Evaluate Model